# Importazione librerie

In [20]:
import pandas as pd
import os

# Importazione e conversione datasets

Data la grande dimensione di alcuni dei dataset utilizzati, questi verranno convertiti da `.csv` a `.parquet` (Apache Parquet Format)

In [21]:
#df = pd.read_csv('./Datasets/Dati_sensori_meteo_2021.csv', low_memory=False)
#df.to_parquet('./Datasets/Dati_sensori_meteo_2021.parquet')

Il codice sotto funziona ma riempie la RAM

In [22]:
#dtypes = {'IdSensore': 'string', 'Valore':'string'}
#parse_dates
#meteo = pd.concat(map(pd.read_parquet, [f'{path}Dati_sensori_meteo_2021_parte_1.parquet', f'{path}Dati_sensori_meteo_2021_parte_2.parquet']))
#meteo.head()

Importazione datasets convertiti

In [42]:
path = './data/'

#### Rilevazioni sensori metereologiche
##### Attributi: 
- IdSensore: 
    - Tipologia: Testo normale
- Data: 
    - Data e ora, Marcatura oraria flessibile
- Valore: 
    - Tipologia: numero
    - Legenda: 9999 = dato mancante 888, 8888 = direzione vento variabile 777, 7777 = calma (solo per direzione di vento)
- idOperatore	
    - *Molto utile per capire la dimensione del dato*
    - LEGENDA: 1: Valore medio 3: Valore massimo 4: Valore cumulato (per la pioggia)
    - Testo normale
- Stato	
    - LEGENDA: VA, VV = dato valido NA, NV, NC = dato invalido NI = dato incerto ND = dato non disponibile
    - Testo normale

In [24]:
#meteo_21 = pd.read_parquet(f'{path}/data_meteo/Dati_sensori_meteo_2021.parquet')
#meteo_21.head()

#### Sensori metereologici
##### Attributi: 
- **IdSensore**
    - Foreign Key con vincolo di integrità referenziale a `IdSensore` di meteo_21
    - Testo normale
- **Tipologia**
    - Grandezza misurata
    - Testo normale
- **Unità DiMisura**
    - Unità di misura della grandezza
    - Testo normale
- IdStazione	
    - Numero identificativo della stazione (penso perchè ogni stazione può avere più sensori)
    - Testo normale
- NomeStazione	
    - Località della stazione (Spesso è 'Comune via' o 'Comune località)'
    - Testo normale
- Quota	
    - Altitudine
    - Numero
- Provincia
    - Sigla della provincia
    - Testo normale
- DataStart	
    - Data e ora
- DataStop	
    - Data e ora
- Storico	
    - Noto che assume valori N e S ma non capisco cosa sia
    - Testo normale
- UTM_Nord	
    - Coordinata UTM nord (le cordinate utm permettono di individuare univocamente un punto sulla cartina terrestre)
    - Testo normale
- UTM_Est	
    - Coordinata UTM est
    - Testo normale
- lng	
    - Longitudine (senza simbolo gradi)
    - Numero
- lat	
    - Latitudine (senza simbolo gradi)
    - Numero
- location	
    - (latitudine°, longitudine°)
    - Posizione
    
##### Grandezze disponibili
Sono disponibili i dati delle grandezze:
- Livello Idrometrico (cm)
- Altezza neve (cm)
- Precipitazione (mm)
- Temperatura (°C)
- Umidità Relativa (%)
- Radiazione Globale (W/m2)
- Velocità e Direzione Vento (m/s e gradi).
- Velocità e direzione del vento (m/s e gradi N) raffica
NB: l’orario del dato è "ora solare" e si riferisce alle osservazioni ottenute fino all’orario indicato.

In [25]:
#sensori_meteo = pd.read_csv(f'{path}Stazioni_Meteorologiche.csv', dtype = {'Tipologia': 'category'})
#sensori_meteo.head()
#stazioni_meteo['Tipologia'].value_counts()

### Rilevazioni sensori qualità dell'aria
##### Attributi: 
- **IdSensore**: 
    - Identificativo Univoco che distingue il sensore
    - Tipologia: Testo normale
- **Data**: 
    - Data e ora, Marcatura oraria flessibile
    - I dati forniti hanno frequenza oraria tranne PM10 e PM2.5 per i quali è fornita la media giornaliera.
- **Valore**: 
    - Tipologia: numero
    - LEGENDA:-9999 = dato mancante o invalido
- **idOperatore**
    - *Molto utile per capire la dimensione del dato*
    - LEGENDA: 1: Valore medio 
    - In realtà: I dati forniti hanno frequenza oraria tranne PM10 e PM2.5 per i quali è fornita la media giornaliera.
    - Testo normale
- **Stato**	
    - VA = dato valido NA = dato invalido I dati presenti in questo archivio relativi agli ultimi 3-6 mesi, contengono ancora valori incerti che possono subire modifiche da parte degli operatori delle reti (invalidazione manuale). Il processo di validazione dei dati prevede una fase di valutazione finale che si conclude entro il 30.3 dell’anno successivo a quello di misura.
    - Testo normale
    
##### Caratteristiche

- La rete di rilevamento della qualità dell’aria di ARPA Lombardia è costituita da stazioni fisse che, per mezzo di analizzatori automatici, forniscono dati in continuo ad intervalli temporali regolari. 
- Le specie di inquinanti monitorate in continuo sono NOX, SO2, CO, O3, PM10, PM2.5 e benzene. A seconda del contesto ambientale nel quale è attivo il monitoraggio, diversa è la tipologia di inquinanti che è necessario rilevare. Pertanto, non tutte le stazioni sono dotate della medesima strumentazione analitica. 
- Le postazioni regionali sono distribuite su tutto il territorio regionale in funzione della densità abitativa e della tipologia di territorio rispettando i criteri definiti dal D.Lgs. 155/2010.
- **I dati forniti hanno frequenza oraria tranne PM10 e PM2.5 per i quali è fornita la media giornaliera.**

In [47]:
# Converto in importazione i -9999 in NA
aria_21 = pd.read_csv(f'{path}/data_aria/Dati_sensori_aria_2021.csv', na_values=['-9999'])
aria_21.shape

(2625796, 5)

In [48]:
# Converto in importazione i -9999 in NA
aria_20 = pd.read_csv(f'{path}/data_aria/Dati_sensori_aria_2020.csv', na_values=['-9999'])
aria_20.shape

(2620280, 5)

In [61]:
aria_20_21 = pd.concat([aria_20, aria_21]).reset_index(drop=True)
aria_20_21.tail()

,IdSensore,Data,Valore,Stato,idOperatore
5246071,10331,19/12/2021 03:00:00 AM,45.7,VA,1
5246072,10333,24/12/2021 07:00:00 PM,39.8,VA,1
5246073,10331,20/12/2021 06:00:00 PM,27.6,VA,1
5246074,10333,25/12/2021 11:00:00 AM,84.7,VA,1
5246075,10331,21/12/2021 04:00:00 AM,34.3,VA,1


### Sensori qualità dell'aria
##### Attributi: 
- IdSensore
    - Foreign Key con vincolo di integrità referenziale a `IdSensore` di aria_21
    - Testo normale
- NomeTipoSensore	
    - Grandezza misurata
    - Testo normale
- Unità DiMisura	
    - Unità di misura della grandezza
    - Testo normale
- IdStazione	
    - Numero identificativo della stazione (penso perchè ogni stazione può avere più sensori)
    - Testo normale
- NomeStazione	
    - Località della stazione (Spesso è 'Comune via' o 'Comune località)'
    - Testo normale
- Quota	
    - Altitudine
    - Numero
- Provincia
    - Sigla della provincia
    - Testo normale
- DataStart	
    - Data e ora
- DataStop	
    - Data e ora
- Storico	
    - Noto che assume valori N e S ma non capisco cosa sia
    - Testo normale
- UTM_Nord	
    - Coordinata UTM nord (le cordinate utm permettono di individuare univocamente un punto sulla cartina terrestre)
    - Testo normale
- UTM_Est	
    - Coordinata UTM est
    - Testo normale
- lng	
    - Longitudine (senza simbolo gradi)
    - Numero
- lat	
    - Latitudine (senza simbolo gradi)
    - Numero
- location	
    - (latitudine°, longitudine°)
    - Posizione

In [56]:
sensori_aria = pd.read_csv(f'{path}Stazioni_qualita_dell_aria.csv', dtype = {'NomeTipoSensore': 'category'})
sensori_aria.tail()
#stazioni_aria['NomeTipoSensore'].value_counts()

,IdSensore,NomeTipoSensore,UnitaMisura,Idstazione,NomeStazione,Quota,Provincia,Comune,Storico,DataStart,DataStop,Utm_Nord,UTM_Est,lat,lng,location
961,6606,Particolato Totale Sospeso,µg/m³,591,Seriate v. Garibaldi,256.0,BG,Seriate,S,20/11/1991,27/01/2000,5059449,555672,45.686356,9.714948,"(45.68635606439914, 9.714947917631529)"
962,6382,Ossidi di Azoto,µg/m³,569,Sondrio v.Mazzini,307.0,SO,Sondrio,N,01/11/1993,NaN,5113078,567173,46.167967,9.870144,"(46.16796681227828, 9.87014407497457)"
963,6607,Particolato Totale Sospeso,µg/m³,591,Seriate v. Garibaldi,256.0,BG,Seriate,S,07/09/2000,04/04/2002,5059449,555672,45.686356,9.714948,"(45.68635606439914, 9.714947917631529)"
964,20523,Ammoniaca,µg/m³,583,Bergamo v.Meucci,249.0,BG,Bergamo,N,NaN,NaN,5059922,550116,45.691037,9.643651,"(45.69103740547214, 9.643650579461385)"
965,12597,Cadmio,ng/m³,609,Casirate d'Adda v. Cimitero,108.0,BG,Casirate d'Adda,N,24/04/2008,NaN,5038450,543458,45.498227,9.556232,"(45.49822713394494, 9.556232262351761)"


In [29]:
sensori_aria.dtypes

IdSensore             int64
NomeTipoSensore    category
UnitaMisura          object
Idstazione            int64
NomeStazione         object
Quota               float64
Provincia            object
Comune               object
Storico              object
DataStart            object
DataStop             object
Utm_Nord              int64
UTM_Est               int64
lat                 float64
lng                 float64
location             object
dtype: object

#### Zonizzazione

In [43]:
zonizzazione = pd.read_csv(f'{path}zonizzazione/zonizzazione_ABCD.csv', dtype = {'Zona': 'category'})
zonizzazione.head()
#sum(zonizzazione['Residenti 2008'])
#zonizzazione['Zona'].value_counts()

,Provincia,Codice Istat,Comune,Zona,Residenti 2008,Superficie (ha)
0,BG,16009,AMBIVERE,A,2341,327
1,BG,16013,ARZAGO D'ADDA,A,2836,944
2,BG,16018,BAGNATICA,A,4119,639
3,BG,16020,BARIANO,A,4396,714
4,BG,16021,BARZANO',A,5178,356


# Analisi esplorativa

- verificare quali siano tutte le quantità uniche rilevate dai sensori qualità dell'aria
- quali siano i gruppi di inquinanti principali
- quali siano i limiti di assunzione umana (gionralieri e annuali)
    - Vedi [qui](https://www.arpalombardia.it/Pages/Aria/Inquinanti.aspx) per limiti e piccola descrizione
    - Vedi [qui](https://www.regione.lombardia.it/wps/portal/istituzionale/HP/DettaglioRedazionale/servizi-e-informazioni/cittadini/salute-e-prevenzione/Sicurezza-negli-ambienti-di-vita-e-di-lavoro/inquinamento-atmosferico/inquinamento-atmosferico/) per approfondimento

In [65]:
sensori_aria['NomeTipoSensore'].value_counts()

Biossido di Azoto             166
Ossidi di Azoto               166
Biossido di Zolfo             125
Monossido di Carbonio         101
Ozono                          91
PM10 (SM2005)                  90
Particolato Totale Sospeso     55
Particelle sospese PM2.5       40
Benzene                        32
Nikel                          15
Piombo                         15
Arsenico                       15
Cadmio                         15
Benzo(a)pirene                 15
Ammoniaca                      14
PM10                            4
BlackCarbon                     4
Monossido di Azoto              3
Name: NomeTipoSensore, dtype: int64

- Si potrebbero considerare solo gli inquinanti **più importanti e con più stazioni**
- Ovvero **Biossido di Azoto | PM10 (SM2005) e PM2,5 e/o Particolato totale | Ozono troposferico | Bisossido di Zolfo**
- Facoltativi perchè non presenti nella valutazione della qualità dell'aria [qui](https://www.arpalombardia.it/Pages/Aria/Modellistica/Indice-qualit%C3%A0-aria.aspx): Monossido di carbonio e benzene

## Missing value e valori non validi

Voglio verificare i *dati invalidi* presenti nei due dataset dei dati
- Solo validi per il meteo

In [52]:
aria_20_21['Valore'].isna().sum()
meteo_21['Stato'].value_counts(dropna = False)

VA    59900066
Name: Stato, dtype: int64

- Un discreto numero di NaN nei dati sulla qualità dell'aria (c'è congruenza tra i NaN di stato e di valore)

In [51]:
aria_20_21['Stato'].value_counts(dropna = False)

VA     5137703
NaN     108373
Name: Stato, dtype: int64

In [55]:
aria_20_21['Valore'].isna().sum()

108373

## Riduzione dimensioni

Meteo: 343 stazioni con 1262 sensori

Aria: 174 stazioni con 966 sensori

Zonizzazione: 9 826 141 abitanti in 1546 comuni divisi in 7 o 8 zone

#### Riduzione dimensioni `aria_20_21` (eventuale `meteo_21`)
##### Rimozione righe mancanti
- L'invalidità del dato è descritta sia dall'attributo `Stato` che dalla presenza di NaN in `Valore`
- MA (VA = dato valido NA = dato invalido I dati presenti in questo archivio relativi agli ultimi 3-6 mesi, contengono ancora valori incerti che possono subire modifiche da parte degli operatori delle reti (invalidazione manuale). Il processo di validazione dei dati prevede una fase di valutazione finale che si conclude entro il 30.3 dell’anno successivo a quello di misura)

In [63]:
aria_20_21.dropna(subset = ['Valore', 'Stato'], inplace = True)
aria_20_21.shape

(5137703, 5)

#### Rimozione sensori di grandezze minori
- da `stazioni_aria` e `aria_20_21` trovare solo gli `IdSensore` di tipologia utile
- Biossido di Azoto | PM10 (SM2005) e PM2,5 e/o Particolato totale | Ozono troposferico | Bisossido di Zolfo 
- Come categorie minori decido di tenere momentaneamente *Monossido di carbonio, benzene, Ossidi di Azoto e PM10*

In [77]:
sensori_aria['NomeTipoSensore'].value_counts()

Biossido di Azoto             166
Ossidi di Azoto               166
Biossido di Zolfo             125
Monossido di Carbonio         101
Ozono                          91
PM10 (SM2005)                  90
Particolato Totale Sospeso     55
Particelle sospese PM2.5       40
Benzene                        32
Nikel                          15
Piombo                         15
Arsenico                       15
Cadmio                         15
Benzo(a)pirene                 15
Ammoniaca                      14
PM10                            4
BlackCarbon                     4
Monossido di Azoto              3
Name: NomeTipoSensore, dtype: int64

In [82]:
grandezze_di_interesse = ['Biossido di Azoto',
 'PM10 (SM2005)',
 'Ossidi di Azoto',
 'Ozono',
 'Biossido di Zolfo',
 'Particolato Totale Sospeso',
 'Monossido di Carbonio',
 'Particelle sospese PM2.5',
 'PM10',
 'Benzene']
sensori_aria.drop(sensori_aria[~sensori_aria['NomeTipoSensore'].isin(grandezze_di_interesse)].index, inplace = True)

## JOIN

Inner join tra `aria_20_21` e `sensori_aria` a dare `aria_e_sensori_20_21`

In [86]:
aria_e_sensori_20_21 = pd.merge(aria_20_21, sensori_aria, how = 'inner', on = 'IdSensore')
aria_e_sensori_20_21.head()

,IdSensore,Data,Valore,Stato,idOperatore,NomeTipoSensore,UnitaMisura,Idstazione,NomeStazione,Quota,Provincia,Comune,Storico,DataStart,DataStop,Utm_Nord,UTM_Est,lat,lng,location
0,6411,01/01/2020 00:00:00,1.0,VA,1,Monossido di Carbonio,mg/m³,627,Cremona P.zza Cadorna,39.0,CR,Cremona,N,12/12/1997,NaN,4998110,579872,45.131947,10.015742,"(45.13194691285173, 10.015741513336042)"
1,6411,01/01/2020 01:00:00,1.0,VA,1,Monossido di Carbonio,mg/m³,627,Cremona P.zza Cadorna,39.0,CR,Cremona,N,12/12/1997,NaN,4998110,579872,45.131947,10.015742,"(45.13194691285173, 10.015741513336042)"
2,6411,01/01/2020 02:00:00,1.1,VA,1,Monossido di Carbonio,mg/m³,627,Cremona P.zza Cadorna,39.0,CR,Cremona,N,12/12/1997,NaN,4998110,579872,45.131947,10.015742,"(45.13194691285173, 10.015741513336042)"
3,6411,01/01/2020 03:00:00,0.9,VA,1,Monossido di Carbonio,mg/m³,627,Cremona P.zza Cadorna,39.0,CR,Cremona,N,12/12/1997,NaN,4998110,579872,45.131947,10.015742,"(45.13194691285173, 10.015741513336042)"
4,6411,01/01/2020 04:00:00,0.9,VA,1,Monossido di Carbonio,mg/m³,627,Cremona P.zza Cadorna,39.0,CR,Cremona,N,12/12/1997,NaN,4998110,579872,45.131947,10.015742,"(45.13194691285173, 10.015741513336042)"


- inner joint tra `aria_xx` e `meteo_xx` con stazioni filtrate
- left joint (non case sensitive) con dataset `zonizzazione` rispetto a `Comune` tenendo solo la colonna `Zona`
- tenere solo stazioni meteo dove ci sono stazioni aria (joint su `lat` e `lng`?)
- salvare i nuovi dataset in formato parquet

In [ ]:
stazioni_aria = sensori_aria.groupby(['NomeStazione', 'Comune'], as_index = False).count()
pd.merge(stazioni_aria, zonizzazione, left_on = stazioni_aria['Comune'].str.lower(), right_on = zonizzazione['Comune'].str.lower(), how='left')